#TTS DECK IMAGE CREATOR
##Prerequisites
1. Place this script in a Google Drive folder with the Google Sheet you want to create.
2. Create a folder within the folder above that you want to export the decklist(s) to.
3. Upload the *Images* folder inside the folder as well.

##Script Instructions
1. Change *FOLDER_LOCATION* to the folder where this script is (keep the /)
2. Change *SHEET_NAME* to the name of the Google Sheet you wish to import from
3. Change *OUTPUT_FOLDER* to the name of your output folder (keep the /)
4. Once you have done all the required setup, your file structure should look like:
  - *FOLDER_LOCATION*
    - *Images*
    - *OUTPUT_FOLDER*
    - *SHEET_NAME*
    - *Generate Images.ipynb*
5. Click *Runtime* at the top, then select *Run all*.
6. You will be asked to sign in to Google. The script will not complete unless you do but if you don't trust my code, now is the time to turn back lmao.
7. It will take a minute or two to resolve, then your deck images will be in *OUTPUT_FOLDER*, ready to import into TTS.

##TTS Import Instructions
1. Go to Objects->Components->Cards->Custom Deck.
2. Place the deck then right click.
3. Choose the image created by this script as the Face Image.
4. Choose any image as the Back. Resize it to 750x1050 for best results.
5. Set Width to 10, Height to 6.
6. Enter deck size. Usually 49 unless you're playing Law.
7. Check *Hidden as Back* to make the cards look like your card back when in your hand.
8. Click Import and wait, it should be done in 10 seconds tops.

In [1]:
# CHANGE THE PARAMETERS HERE BEFORE RUNNING
FOLDER_LOCATION = "/content/drive/MyDrive/AU/"
SHEET_NAME = "AU_Decklists_V0.5.6"
OUTPUT_FOLDER = "Deck_TTS_Images/"

In [7]:
# Required Installs
!pip install --upgrade -q gspread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.2 MB/s 

In [3]:
# Required Libraries
import gspread
import os
import pandas as pd
import PIL as pil
from google.colab import auth
auth.authenticate_user()
from google.auth import default
from google.colab import drive

# Open the Google Sheet
drive.mount("/content/drive/")
creds, _ = default()
gc = gspread.authorize(creds)
worksheets = gc.open(SHEET_NAME)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# Get all the card images
image_dict = {}
for image_name in os.listdir(FOLDER_LOCATION + "Images"):
  image_dict[int(image_name[str.rindex(image_name, "_")+1:str.rindex(image_name,".")])] = pil.Image.open(FOLDER_LOCATION + "Images/" + image_name)

In [5]:
# Get the sheet names from the Google Sheet (used for deck names)
worksheet_metadata = worksheets.fetch_sheet_metadata()['sheets']
keys = []
for i in range(len(worksheet_metadata)):
  keys.append(worksheet_metadata[i]['properties']['title'])

In [6]:
# Create the deck images
for i in range(len(keys)): # for each Sheet create one deck
  rows = worksheets.get_worksheet(i).get_all_values()
  card_ids = []
  for j in range(1, len(rows)): # Get the ID of each card in the deck
    if rows[j][1]:
      for k in range(int(rows[j][0])):
        card_ids.append(j-1)
  output = pil.Image.new("RGB",(7500, 6300)) # Create output image
  for j in range(len(card_ids)): # Paste the card images into the correct locations
    offset = ((j * 750) % 7500, (j // 10) * 1050)
    output.paste(image_dict[card_ids[j]], offset)
  output.save(FOLDER_LOCATION + OUTPUT_FOLDER + keys[i] + ".png") # Save the final image